<a href="https://colab.research.google.com/github/Huypham07/LLM-News-Crawler/blob/main/llm_model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Đánh giá các mô hình LLM

## Mục tiêu
* So sánh hiệu suất giữa các mô hình LLM:
  * Gemini
  * Mistral
  * Llama
  * Gemma
* Kết luận được mô hình phù hợp cho hệ thống Crawler
* Bên cạnh đó đánh giá phương pháp lọc đầu vào để giảm input token

## Chuẩn bị tập test

In [342]:
import requests
from bs4 import BeautifulSoup

In [343]:
urls = [
    'https://vnexpress.net/khong-quan-ukraine-tuyen-bo-ban-ha-tiem-kich-su-35-nga-4895862.html',
    'https://dantri.com.vn/xa-hoi/bo-chinh-tri-sap-nhap-tinh-dong-thoi-voi-hop-nhat-xa-o-noi-du-dieu-kien-20250607170102455.htm',
    'https://thanhnien.vn/dang-nha-nuoc-luon-quan-tam-cong-dong-nguoi-viet-o-nuoc-ngoai-185250607171121924.htm',
    'https://tienphong.vn/nhan-xet-giai-de-de-ngu-van-ha-noi-de-tho-nhung-kho-dat-diem-9-post1749112.tpo'
]

raw_htmls = []
for url in urls:
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  html = soup.prettify()
  raw_htmls.append(html)

## Chuẩn bị mô hình

In [344]:
import json
import time
from openai import OpenAI
from google import genai

from google.colab import userdata
open_router_api_key = userdata.get('OPEN_ROUTER_API_KEY')
google_api_key = userdata.get('GOOGLE_API_KEY')

In [345]:
models = {
    "mistralai/devstral-small:free": "mistralai/Devstral-Small-2505",
    "meta-llama/llama-3.3-70b-instruct:free": "meta-llama/Llama-3.3-70B-Instruct",
    "google/gemma-3-27b-it:free": "google/gemma-3-27b-it",
    "gemini-2.0-flash-lite-001": "gemini-2.0-flash-lite",
    "gemini-2.0-flash-001": "gemini-2.0-flash",
    "gemini-2.5-flash-preview-05-20": "gemini-2.5-flash-preview",
}

model_names = {
    "mistralai/devstral-small:free": "Devstral-Small",
    "meta-llama/llama-3.3-70b-instruct:free": "Llama-3.3-70B",
    "google/gemma-3-27b-it:free": "Gemma-3-27b",
    "gemini-2.0-flash-lite-001": "Gemini-2.0-flash-lite",
    "gemini-2.0-flash-001": "Gemini-2.0-flash",
    "gemini-2.5-flash-preview-05-20": "Gemini-2.5-flash-preview",
}

In [346]:
def build_prompt(html: str):
  return f"""You are a CSS selector expert. Analyze this cleaned HTML from a news website and return ONLY a JSON object with CSS selectors.

HTML: {html}

Return JSON with these exact keys:
{{
  "title": "best CSS selector for article title",
  "content": "best CSS selector for main article content",
  "author": "best CSS selector for author name",
  "date": "best CSS selector for publish date"
}}

Rules:
- Use null if element not found
- Prefer class/id selectors over complex hierarchies
- Choose selectors that would work for similar pages
- No explanations, only JSON"""

In [347]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=open_router_api_key,
)

gemini_client = genai.Client(api_key=google_api_key)

In [348]:
import regex

def extract_outermost_json_block(text: str) -> str:
    match = regex.search(r'\{(?:[^{}]|(?R))*\}', text.strip())
    return match.group(0) if match else ''

In [349]:
def call_model(model, prompt):
  start_time = time.perf_counter()
  if model.startswith("gemini-"):
    response = gemini_client.models.generate_content(
      model=model,
      contents=prompt,
    )
    json_str = extract_outermost_json_block(response.text)
  else:
    completion = client.chat.completions.create(
      extra_body={},
      model=model,
      messages=[
        {
          "role": "user",
          "content": prompt
        }
      ]
    )
    json_str = extract_outermost_json_block(completion.choices[0].message.content)


  end_time = time.perf_counter()
  elapsed_time = end_time - start_time  # in seconds
  print(json_str)
  result = json.loads(json_str)
  return {
    **result,
    "time_res": f"{round(elapsed_time * 1000)}ms"
  }


In [350]:
def compare_models(html):
  prompt = build_prompt(html)
  results = {}
  for model in models.keys():
    print(f"Querying {model}...")
    output = call_model(model, prompt)
    results[model] = output
  return results

## Kiểm thử

### Tối ưu input token bằng cách lọc raw HTML, giữ lại các thẻ cần thiết

In [351]:
# filter
filtered_htmls = []
remove_elements = [
    "script", "noscript", "style", "head", "footer", "nav", "[class*='menu']", "[class*='nav']",
    "svg", "img", "link", ".ads", ".advertisment", ".social", "[class*='banner']",
    "[id*='ads']", "[id*='banner']", "[class*='share']", "[id*='share']", "[class*='comment']",
    "[id*='comment']", ".copy-right", "#copy-right"
]

for html in raw_htmls:
    soup = BeautifulSoup(html, "html.parser")

    # xóa các selector thông thường
    for selector in remove_elements:
        for tag in soup.select(selector):
            tag.decompose()

    # xóa <header> nếu không có <h1> bên trong
    for header_tag in soup.find_all("header"):
        if not header_tag.find("h1"):
            header_tag.decompose()

    filtered_htmls.append(soup.prettify())


In [352]:
import tiktoken
from transformers import AutoTokenizer
import pandas as pd
from IPython.display import clear_output

In [353]:
# create HF_TOKEN in Colab secrets to use AutoTokenizer
from huggingface_hub import login
hf_token = userdata.get('HF_TOKEN')
login(token=hf_token)

In [354]:
def count_tokens(model, content):
  try:
      tokenizer = AutoTokenizer.from_pretrained(model)
      return len(tokenizer.encode(content))
  except Exception:
    print(f"Warning: {model} not found. Using cl100k_base encoding.")
    return len(tiktoken.get_encoding("cl100k_base").encode(content))

In [355]:
tokens_before = pd.DataFrame({"Model": models.values()})
tokens_before.index += 1
tokens_before.index.name = "STT"

for i, raw_html in enumerate(raw_htmls, start=1):
    filtered_html = filtered_htmls[i-1]
    token_pairs = [
        count_tokens(model, raw_html)
        for model in models.values()
    ]
    tokens_before[f"Content {i}"] = token_pairs

clear_output(wait=True)
print("Tokens of content before filtering")
print("-----------------")
display(tokens_before)


Tokens of content before filtering
-----------------


,Model,Content 1,Content 2,Content 3,Content 4
STT,,,,,
1,mistralai/Devstral-Small-2505,74601,148634,216730,28946
2,meta-llama/Llama-3.3-70B-Instruct,74601,148634,216730,28946
3,google/gemma-3-27b-it,96190,187469,284612,28555
4,gemini-2.0-flash-lite,74601,148634,216730,28946
5,gemini-2.0-flash,74601,148634,216730,28946
6,gemini-2.5-flash-preview,74601,148634,216730,28946


In [356]:
tokens_after = pd.DataFrame({"Model": models.values()})
tokens_after.index += 1
tokens_after.index.name = "STT"

for i, filtered_html in enumerate(filtered_htmls, start=1):
    token_pairs = [
        count_tokens(model, filtered_html)
        for model in models.values()
    ]
    tokens_after[f"Content {i}"] = token_pairs

clear_output(wait=True)
print("Tokens of content after filtering")
print("-----------------")
display(tokens_after)


Tokens of content after filtering
-----------------


,Model,Content 1,Content 2,Content 3,Content 4
STT,,,,,
1,mistralai/Devstral-Small-2505,3633,7419,8446,12509
2,meta-llama/Llama-3.3-70B-Instruct,3633,7419,8446,12509
3,google/gemma-3-27b-it,3704,7226,8218,11101
4,gemini-2.0-flash-lite,3633,7419,8446,12509
5,gemini-2.0-flash,3633,7419,8446,12509
6,gemini-2.5-flash-preview,3633,7419,8446,12509


In [357]:
token_reduction_percent = pd.DataFrame({"Model": models.values()})
token_reduction_percent.index += 1
token_reduction_percent.index.name = "STT"

# ((before - after) / before) * 100
for col in tokens_before.columns[1:]:
    before = tokens_before[col]
    after = tokens_after[col]
    reduction_percent = ((before - after) / before * 100).round(2)
    token_reduction_percent[col] = reduction_percent.astype(str) + '%'

clear_output(wait=True)
print("Token reduction percentage after filtering")
print("------------------------------------------")
display(token_reduction_percent)


Token reduction percentage after filtering
------------------------------------------


,Model,Content 1,Content 2,Content 3,Content 4
STT,,,,,
1,mistralai/Devstral-Small-2505,95.13%,95.01%,96.1%,56.79%
2,meta-llama/Llama-3.3-70B-Instruct,95.13%,95.01%,96.1%,56.79%
3,google/gemma-3-27b-it,96.15%,96.15%,97.11%,61.12%
4,gemini-2.0-flash-lite,95.13%,95.01%,96.1%,56.79%
5,gemini-2.0-flash,95.13%,95.01%,96.1%,56.79%
6,gemini-2.5-flash-preview,95.13%,95.01%,96.1%,56.79%


### Thực hiện kiểm thử các mô hình

In [362]:
# urls = [
#     'https://vnexpress.net/khong-quan-ukraine-tuyen-bo-ban-ha-tiem-kich-su-35-nga-4895862.html',
#     'https://dantri.com.vn/xa-hoi/bo-chinh-tri-sap-nhap-tinh-dong-thoi-voi-hop-nhat-xa-o-noi-du-dieu-kien-20250607170102455.htm',
#     'https://thanhnien.vn/dang-nha-nuoc-luon-quan-tam-cong-dong-nguoi-viet-o-nuoc-ngoai-185250607171121924.htm',
#     'https://tienphong.vn/nhan-xet-giai-de-de-ngu-van-ha-noi-de-tho-nhung-kho-dat-diem-9-post1749112.tpo'
# ]
expect_results = [
    {
      "title": "h1.title-detail",
      "content": "article.fck_detail",
      "author": "p.Normal strong",
      "date": "span.date"
    },
    {
      "title": "h1.title-page detail",
      "content": "div.singular-content",
      "author": ".author-name b",
      "date": "time.author-time"
    },
    {
      "title": "h1.detail-title > span[data-role='title']",
      "content": "div.detail-content",
      "author": "div.author-info a.name",
      "date": "div.detail-time div[data-role='publishdate']"
    },
    {
      "title": "h1.article__title",
      "content": "div.article__body",
      "author": ".article__author .name",
      "date": ".article__meta time"
    },
]

In [359]:
results_multi_model = []

for html in filtered_htmls:
  results_multi_model.append(compare_models(html))

clear_output(wait=True)
print(len(results_multi_model))

4


In [363]:
fields = ["title", "content", "author", "date", "time_res"]
for i, exp_result in enumerate(expect_results, start=1):
  data = {"Field": fields, "Reality Result": [exp_result.get(f, "") if f != "time_res" else "" for f in fields]}
  for model_name, result in results_multi_model[i-1].items():
    data[model_names[model_name]] = [result.get(f, "") for f in fields]
  df = pd.DataFrame(data)
  df.set_index("Field", inplace=True)
  print("---------------------")
  print(f"Content {i}")
  display(df)

---------------------
Content 1


,Reality Result,Devstral-Small,Llama-3.3-70B,Gemma-3-27b,Gemini-2.0-flash-lite,Gemini-2.0-flash,Gemini-2.5-flash-preview
Field,,,,,,,
title,h1.title-detail,.title-detail,.title-detail,h1.title-detail,.title-detail,.title-detail,h1.title-detail
content,article.fck_detail,.fck_detail,.fck_detail,article.fck_detail p.Normal,.fck_detail,.fck_detail,article.fck_detail
author,p.Normal strong,None,p.Normal strong,p.Normal strong,.fck_detail p strong,p.Normal[style='text-align:right;'] > strong,article.fck_detail p.Normal strong
date,span.date,.date,.date,span.date,.header-content .date,.date,span.date
time_res,,2039ms,1549ms,4258ms,862ms,758ms,6251ms


---------------------
Content 2


,Reality Result,Devstral-Small,Llama-3.3-70B,Gemma-3-27b,Gemini-2.0-flash-lite,Gemini-2.0-flash,Gemini-2.5-flash-preview
Field,,,,,,,
title,h1.title-page detail,h1.title-page.detail,.title-page.detail,article.singular-container h1.title-page.detail,h1.title-page.detail,.title-page.detail,h1.title-page.detail
content,div.singular-content,div.singular-content,.singular-content,article.singular-container div.singular-content,div.singular-content,.singular-content,div.singular-content
author,.author-name b,div.author-name > a > b,.author-name b,div.author-meta a b,div.author-name a b,.author-name a b,.author-name b
date,time.author-time,time.author-time,.author-time,div.author-meta time.author-time,time.author-time,.author-time,time.author-time
time_res,,3109ms,1844ms,4818ms,834ms,745ms,3103ms


---------------------
Content 3


,Reality Result,Devstral-Small,Llama-3.3-70B,Gemma-3-27b,Gemini-2.0-flash-lite,Gemini-2.0-flash,Gemini-2.5-flash-preview
Field,,,,,,,
title,h1.detail-title > span[data-role='title'],.detail-title span[data-role='title'],.detail-title span[data-role='title'],h1.detail-title span[data-role='title'],".detail-title > span[data-role=""title""]",".detail-title > span[data-role=""title""]",h1.detail-title
content,div.detail-content,.detail-cmain div[data-role='content'],.detail-content,.detail-content[data-role='content'],".detail-content.afcbc-body[data-role=""content""]",".detail-content[data-role=""content""]",div.detail-content
author,div.author-info a.name,.detail-author a[name],.author-info .name,.detail-author a[href*='/author/'],.detail-author .name,.detail-author .name,div.detail-author a.name
date,div.detail-time div[data-role='publishdate'],.detail-time div[data-role='publishdate'],[data-role='publishdate'],.detail-time div[data-role='publishdate'],".detail-time [data-role=""publishdate""]",".detail-time [data-role=""publishdate""]","div[data-role=""publishdate""]"
time_res,,2374ms,2005ms,5344ms,1026ms,849ms,3101ms


---------------------
Content 4


,Reality Result,Devstral-Small,Llama-3.3-70B,Gemma-3-27b,Gemini-2.0-flash-lite,Gemini-2.0-flash,Gemini-2.5-flash-preview
Field,,,,,,,
title,h1.article__title,.article__title.cms-title,.article__title,h1.article__title.cms-title,h1.article__title.cms-title,.article__title.cms-title,h1.article__title
content,div.article__body,.article__body.cms-body,.article__body,div.article__body.cms-body,div.article-content,.article__body.cms-body,div.article__body
author,.article__author .name,.cms-author,.article__author .name,span.cms-author,span.name.cms-author,.article__author .name.cms-author,span.cms-author
date,.article__meta time,time[datetime],.article__meta .time time,time[datetime],time[datetime],.article__meta .time time,span.time time
time_res,,3476ms,2360ms,5227ms,949ms,768ms,5279ms


## Đánh giá & Kết luận

### Đối với filtering HTML

Kết quả đa số đều ở mức khá tốt > 90% nội dung không cần thiết bị loại bỏ đi. Đối với một số trang web có cấu trúc đặc biệt hơn thì vẫn đảm bảo loại bỏ được hơn 50% nội dung không cần thiết, đảm bảo được token đầu vào ở mức ~ 15000 hoặc ít hơn, dùng được cho các model thử nghiệm trong bài này.

### Đối với Kết quả CSS Selector thu được của các model

| Model                        | Title                          | Content                        | Author                         | Date                           | Thời gian phản hồi |
|-----------------------------|--------------------------------|--------------------------------|--------------------------------|--------------------------------|---------------------|
| Mistral/Devstral-Small      | Gần như chính xác   | Gần như chính xác   | Độ chính xác chưa cao          | Gần như chính xác   | 2–3s               |
| Llama-3.3-70B               | Gần như chính xác              | Gần như chính xác              | Gần như chính xác              | Gần như chính xác              | 1–2s               |
| Gemma-3-27b                 | Chi tiết, gần như chính xác    | Chi tiết, gần như chính xác    | Chi tiết, gần như chính xác    | Chi tiết, gần như chính xác    | 4–5s               |
| Gemini-2.0-flash-lite       | Gần như chính xác              | Gần như chính xác              | Gần như chính xác              | Gần như chính xác              | ~1s                |
| Gemini-2.0-flash            | Gần như chính xác              | Gần như chính xác              | Gần như chính xác              | Gần như chính xác              | < 1s                |
| Gemini-2.5-flash-preview    | Chính xác gần như hoàn toàn     | Chính xác gần như hoàn toàn    | Chính xác gần như hoàn toàn    | Chính xác gần như hoàn toàn    | 3–6s               |


Từ kết quả trên ta thấy rằng:
* Mô hình mistral có vẻ không phù hợp với bài toán Crawler này do độ chính xác chưa đạt kì vọng cũng như thời gian phản hồi chưa phải quá tốt.
* Mô hình Gemma 3 mặc dù độ chính xác rất cao, chi tiết, nhưng do đây mới chỉ là bài test nhỏ nên hiện tượng quá chi tiết này có thể sẽ dẫn đến lỗi nhiều trong tương lai, do cùng một domain có thể sẽ được thêm, bớt các CSS selector khác nhau. Và thời gian phản hồi của mô hình này cũng là một điểm trừ.
* Mô hình Gemini 2.5 Flash mặc dù kết quả rất tốt nhưng thời gian phản hồi cũng như giới hạn lượng token đầu vào và lượng request trong 1 phút, trong 1 ngày cũng sẽ là lý do ta không lựa chọn cho bài toán này.
* Về Mô hình Llama 3.3, Gemini 2.0 Flash, Gemini 2.0 Flash lite kết quả đều rất tốt, thời gian phản hồi đủ nhanh. Cả ba mô hình này đều có thể dùng được cho bài toán lần này

⇒ Với free tier, ta sẽ sử dụng Gemini 2.0 Flash lite.